In [ ]:
// import { ChatOpenAI } from "@langchain/openai";
import { PromptTemplate } from "@langchain/core/prompts";

// const llm = new ChatOpenAI({
//   modelName: process.env.MODEL_NAME,
//   configuration: {
//     baseURL: process.env.BASE_URL,
//     apiKey: process.env.OPENAI_API_KEY,
//   },
// });

// const prompt = new PromptTemplate({
//     inputVariables: ['job', 'question'],
//     template: '你是一个{job}，请解释一下{question}'
// })
const prompt = PromptTemplate.fromTemplate(
  "你是一个{job}，请回答一下{question}"
);

// await prompt.format({ job: '三甲医院的骨科医生', question: '如何预防半月板损伤' })
// const partial = await prompt.partial({ job: '三甲医院的骨科医生' })

const getJob = (hospital: string) => {
  const date = new Date().toLocaleDateString();
  return hospital + "的骨科医生，今天是" + date;
};
const partial = await prompt.partial({ job: getJob("三甲医院") });

const rst1 = await partial.format({ question: "如何预防半月板损伤" });
console.log(rst1);

const rst2 = await partial.format({ question: "如何处理半月板损伤" });
console.log(rst2);


In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import {
  ChatPromptTemplate,
  HumanMessagePromptTemplate,
  SystemMessagePromptTemplate,
} from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { RunnableSequence } from '@langchain/core/runnables'

const llm = new ChatOpenAI({
  modelName: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const humanTemplate = "{humanText}";
const systemTemplate = "你是一个{systemText}";
// const humanMessage = HumanMessagePromptTemplate.fromTemplate(humanTemplate)
// const systemMessage = SystemMessagePromptTemplate.fromTemplate(systemTemplate)
// const prompt = ChatPromptTemplate.fromMessages([systemMessage, humanMessage])

const prompt = ChatPromptTemplate.fromMessages([
  ["system", systemTemplate],
  ["human", humanTemplate],
]);

// const message1 = await prompt.format({ humanText: '你好', systemText: '三甲医院的主任医生' })
// console.log('message1---', message1)

// const message2 = await prompt.invoke({ humanText: '你好', systemText: '三甲医院的主任医生' })
// console.log('message2---', message2)

const message3 = await prompt.formatMessages({
  humanText: "你好",
  systemText: "三甲医院的主任医生",
});
// console.log("message3---", message3);

const outputParse = new StringOutputParser();

// await llm.pipe(outputParse).invoke(message3)

// const chain = prompt.pipe(llm).pipe(outputParse);
// await chain.invoke({ humanText: "你好", systemText: "三甲医院的主任医生" });

const chain = RunnableSequence.from([prompt, llm, outputParse]);
await chain.invoke({ humanText: "你好", systemText: "三甲医院的主任医生" });


In [ ]:
import {
  PromptTemplate,
  PipelinePromptTemplate,
} from "@langchain/core/prompts";

const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

const fullPrompt = PromptTemplate.fromTemplate(`
你是一个智能管家，今天是 {date}，你的主人的信息是{info}, 
根据上下文，完成主人的需求
{task}
`);

const datePrompt = PromptTemplate.fromTemplate("{date}，现在是 {period}，家里已经备好了{food}")
const periodPrompt = await datePrompt.partial({ date: getCurrentDateStr })

const infoPrompt = PromptTemplate.fromTemplate('姓名是 {name}，性别是 {gender}，截止 {date} 年龄是 {age} 岁')

const taskPrompt = PromptTemplate.fromTemplate(`
我想吃 {period} 的 {food}。 
再重复一遍我的信息 {info}
`)


const composedPrompt = new PipelinePromptTemplate({
    pipelinePrompts: [
        {
            name: 'date',
            prompt: periodPrompt
        }, {
            name: 'info',
            prompt: infoPrompt
        }, {
            name: 'task',
            prompt: taskPrompt
        }
    ],
    finalPrompt: fullPrompt
})

await composedPrompt.format({
    period: '早上',
    name: '张三',
    gender: '男',
    food: '麦当劳',
    age: 16
})
